In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
import os
os.listdir("./utils") 
from utils.monodepth import load_model, depth_map_from_frame
# Load the model
dpt_model, transform, device = load_model()
import numpy as np


/home/hassaan/Downloads/deployement/LiDAR-Pose-Detection/deploymentEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hassaan/Downloads/deployement/LiDAR-Pose-Detection/deploymentEnv/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [17]:
def apply_pose_detection(frame, model):

    # Convert the frame to a format suitable for the model
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get the depth map
    depth_map = depth_map_from_frame(img_rgb, dpt_model, transform, device)
    print("depth map formed")
    cv2.imwrite("output.jpg",depth_map)
    
    depth_map = cv2.imread("output.jpg")    
    
    num_people = 0
    # Perform inference
    results = model.predict(depth_map)
    
    keypoints = results[0].keypoints
    
    if len(keypoints) > 0:
        keypoints = keypoints[0].cpu().numpy()
    else:
        keypoints = None
    # frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    if keypoints is not None:
        xy = keypoints.data[0]
        conf = keypoints.conf[0]
        for i in range(len(xy)):
            x, y, c = xy[i]
            if c > 0:
                cv2.circle(depth_map, (int(x), int(y)), 2, (0, 255, 0), -1)
                cv2.putText(
                    depth_map,
                    f"{i}-{int(c*100)}",
                    (int(x), int(y) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 0, 0),
                    1,
                )
                
    while True:
        cv2.imshow("C", depth_map)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    cv2.destroyAllWindows()
    
    return 123, depth_map
    # return num_people, frame

In [18]:
model = YOLO("models/yolov8m-pose.pt")

In [19]:
ret = True
frame = cv2.imread("/home/hassaan/Downloads/train_2500/images_final/17.png")


if ret:
    num_people, processed_frame = apply_pose_detection(frame, model)
    print(f"Number of detected people: {num_people}")

    # # Display the processed frame
    # cv2.imshow('Processed Frame', processed_frame)
    # cv2.waitKey(0)

# cap.release()
cv2.destroyAllWindows()


0: 384x640 3 persons, 1288.2ms
Speed: 4.8ms preprocess, 1288.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Number of detected people: 123
